# Grover's Algorithm for Finding Minimum Value in an Unsorted Array

This code is a Jupyter Notebook implementing the Grover's algorithm for searching an unsorted array of numbers. 

1. The first cell imports necessary libraries, including NumPy and Qiskit.
2. The second cell defines two functions: `find_min` and `grover_search`. The `find_min` function uses the `grover_search` function to find the minimum value in an unsorted array. The `grover_search` function implements Grover's algorithm, which is a quantum algorithm for searching an unsorted array.
3. The third cell creates an unsorted array of 16 numbers and shuffles it. It then calls the `find_min` function with this array and 64 iterations to find the minimum value.

Grover's algorithm is a quantum algorithm that can search an unsorted array in O(sqrt(N)) time, where N is the size of the array. This is much faster than classical algorithms for searching an unsorted array, which have a time complexity of O(N).

In this code, the `grover_search` function uses Qiskit's implementation of Grover's algorithm to find the index of the minimum value in the array. The function first creates an oracle circuit that marks the indices of the elements in the array that are less than or equal to a given threshold. It then applies the Grover operator to this oracle circuit, which amplifies the probability of finding the minimum value. Finally, it measures the output of the quantum circuit and returns the index of the minimum value.

Overall, this code demonstrates how to implement Grover's algorithm using Qiskit and Python.

In [2]:
import numpy as np
from qiskit import QuantumCircuit, Aer, execute
from qiskit.circuit.library import GroverOperator
import random

In [4]:
# numbers = unsorted array of numbers
def find_min(numbers, iterations):
    n_items = len(numbers)
    threshold = numbers[random.randint(0, n_items-1)] # Choosing a random threshold value
    candidate_indices = [i for i in range(n_items) if numbers[i] <= threshold] # Marking elements of numbers that are <= threshold
    for _ in range(iterations):
        new_threshold = numbers[grover_search(candidate_indices, n_items)] # Grover search to find element smaller than current threshold
        if new_threshold < threshold:
            threshold = new_threshold
            candidate_indices = [idx for idx in candidate_indices if numbers[idx] <= threshold]
    return threshold

def grover_search(candidate_indices, n_items):
    n_qubits = int(np.ceil(np.log2(n_items)))
    oracle = np.identity(2**n_qubits)
    for idx in candidate_indices:
        oracle[idx, idx] = -1 # Building oracle
    oracle_circuit = QuantumCircuit(n_qubits)
    oracle_circuit.unitary(oracle, range(n_qubits))
    grover_op = GroverOperator(oracle_circuit, insert_barriers=True)
    grover_circuit = QuantumCircuit(n_qubits, n_qubits)
    grover_circuit.h(range(n_qubits)) # Initializing equal superposition
    n_grover_iterations = int(np.sqrt(n_items))
    for _ in range(n_grover_iterations):
        grover_circuit = grover_circuit.compose(grover_op)
    grover_circuit.measure(range(n_qubits), range(n_qubits))
    backend = Aer.get_backend("qasm_simulator")
    grover_circuit.decompose().draw()
    job = execute(grover_circuit, backend)
    result = int(list(job.result().get_counts().keys())[0], 2)
    return result

In [8]:
unsorted_numbers = list(range(16))
unsorted_numbers.reverse()
unsorted_numbers.remove(0)  # Remove zero to avoid trivial case
unsorted_numbers.append(10)  # Append a number to ensure the list is not empty
random.shuffle(unsorted_numbers)  # Shuffle the list to create an unsorted array
print(unsorted_numbers)
find_min(unsorted_numbers, 64)

[12, 2, 3, 6, 5, 4, 10, 1, 15, 13, 11, 8, 9, 14, 10, 7]


2